In [21]:
import importlib
import os
from pathlib import Path
import sys
from urllib.parse import urlparse

from arcgis.features import GeoAccessor, FeatureLayer
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import numpy as np

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent
dir_data = dir_prj/'data'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import rec_data

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=None if len(os.getenv('ESRI_GIS_PASSWORD')) is 0 else os.getenv('ESRI_GIS_PASSWORD')
)

gis

GIS @ https://knu2xs.maps.arcgis.com version:9.1

In [3]:
map_itm_id = '313923e7f415433594587911a3df3a39'

admin_dict = {
    'gis.blm.gov': 'BLM',
    'mapservices.nps.gov': 'NPS',
    'apps.fs.usda.gov': 'USFS',
    'gismanager.rco.wa.gov': 'WA'
}

In [4]:
itm = gis.content.get(map_itm_id)

wm = WebMap(itm)

lyr_set = set((lyr.title, lyr.url) for lyr in wm.layers)

lyr_df = pd.DataFrame(lyr_set, columns=['name', 'url']).sort_values('name').reset_index(drop=True)

lyr_df.head()

,name,url
0,BLM Land Ownership,https://gis.blm.gov/arcgis/rest/services/lands...
1,BLM Roads Managed for Limited Public Motorized...,https://gis.blm.gov/arcgis/rest/services/trans...
2,BLM Roads Managed for Public Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...
3,BLM Trails Managed for Limited Public Motorize...,https://gis.blm.gov/arcgis/rest/services/trans...
4,BLM Trails Managed for Public,https://gis.blm.gov/arcgis/rest/services/trans...


In [5]:
lyr_df['admin_agency'] = lyr_df.url.apply(lambda url: admin_dict[urlparse(url).netloc])

lyr_df

,name,url,admin_agency
0,BLM Land Ownership,https://gis.blm.gov/arcgis/rest/services/lands...,BLM
1,BLM Roads Managed for Limited Public Motorized...,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
2,BLM Roads Managed for Public Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
3,BLM Trails Managed for Limited Public Motorize...,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
4,BLM Trails Managed for Public,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
5,BLM Trails Managed for Public Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
6,BLM Trails Managed for Public Non-Mechanized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
7,BLM Trails Managed for Public Non-Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
8,BLM Trails Not Assessed for Public,https://gis.blm.gov/arcgis/rest/services/trans...,BLM
9,US NPS Public Roads - NPS Public Roads,https://mapservices.nps.gov/arcgis/rest/servic...,NPS


In [7]:
def get_category(val):
    val = val.lower()
    if ('road' in val) or ('motorized' in val):
        cat = 'road'
    elif 'trail' in val:
        cat = 'trail'
    elif ('boundary' in val) or ('wilderness' in val) or ('land' in val):
        cat = 'area'
    else:
        cat = np.Nan
    return cat

lyr_df['layer_type'] = lyr_df.name.apply(lambda val: get_category(val))

lyr_df

,name,url,admin_agency,layer_type
0,BLM Land Ownership,https://gis.blm.gov/arcgis/rest/services/lands...,BLM,area
1,BLM Roads Managed for Limited Public Motorized...,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,road
2,BLM Roads Managed for Public Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,road
3,BLM Trails Managed for Limited Public Motorize...,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,road
4,BLM Trails Managed for Public,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,trail
5,BLM Trails Managed for Public Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,road
6,BLM Trails Managed for Public Non-Mechanized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,trail
7,BLM Trails Managed for Public Non-Motorized Use,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,road
8,BLM Trails Not Assessed for Public,https://gis.blm.gov/arcgis/rest/services/trans...,BLM,trail
9,US NPS Public Roads - NPS Public Roads,https://mapservices.nps.gov/arcgis/rest/servic...,NPS,road


In [22]:
usfs_trl_url = lyr_df[(lyr_df.admin_agency == 'USFS') & (lyr_df.layer_type == 'trail')].iloc[0].url
usfs_trl_df = FeatureLayer(usfs_trl_url).query().sdf

print(usfs_trl_df.info())
usfs_trl_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   TRAIL_NO                        2413 non-null   object  
 1   TRAIL_NAME                      1961 non-null   object  
 2   TRAIL_TYPE                      2413 non-null   object  
 3   TRAIL_CN                        2413 non-null   object  
 4   BMP                             2413 non-null   float64 
 5   EMP                             2413 non-null   float64 
 6   SEGMENT_LENGTH                  2413 non-null   float64 
 7   ADMIN_ORG                       2413 non-null   object  
 8   MANAGING_ORG                    2413 non-null   object  
 9   SECURITY_ID                     2413 non-null   object  
 10  ATTRIBUTESUBSET                 2413 non-null   object  
 11  NATIONAL_TRAIL_DESIGNATION      2413 non-null   int64   
 12  TRAIL_CLASS         

,TRAIL_NO,TRAIL_NAME,TRAIL_TYPE,TRAIL_CN,BMP,EMP,SEGMENT_LENGTH,ADMIN_ORG,MANAGING_ORG,SECURITY_ID,...,NONMOTOR_WATERCRAFT_MANAGED,NONMOTOR_WATERCRAFT_ACCPT_DISC,NONMOTOR_WATERCRAFT_RESTRICTED,GIS_MILES,TERRA_MOTORIZED,SNOW_MOTORIZED,WATER_MOTORIZED,OBJECTID,SHAPE_Length,SHAPE
0,18EV412,None,TERRA,323250010602,0.0,1.769,1.769,051654,051654,0516,...,N/A,N/A,N/A,1.722,N/A,N/A,N/A,22,3605.892883,"{""paths"": [[[-13353048.1357, 4564446.605400003..."
1,17EV401,None,TERRA,323251010602,0.0,1.141,1.141,051654,051654,0516,...,N/A,N/A,N/A,1.180,N/A,N/A,N/A,23,2330.534785,"{""paths"": [[[-13365078.46, 4564799.013599999],..."
2,18EV409,None,TERRA,323248010602,0.0,1.026,1.026,051654,051654,0516,...,N/A,N/A,N/A,0.897,N/A,N/A,N/A,24,1840.112250,"{""paths"": [[[-13358913.8565, 4564884.203500003..."
3,18DC485,None,TERRA,322421010602,0.0,0.058,0.058,051651,051651,0516,...,N/A,N/A,N/A,0.017,N/A,N/A,N/A,25,37.162609,"{""paths"": [[[-13362515.2314, 4605467.162699997..."
4,18EV408,None,TERRA,323247010602,0.0,0.163,0.163,051654,051654,0516,...,N/A,N/A,N/A,0.163,N/A,N/A,N/A,26,333.114299,"{""paths"": [[[-13359826.808400001, 4564428.2088..."


In [23]:
usfs_trl_df.TRAIL_TYPE.unique()

array(['TERRA'], dtype=object)

In [30]:
r = usfs_trl_df[usfs_trl_df.TRAIL_NAME.notnull()].iloc[0]

In [42]:
usfs_trl_df.columns.to_frame()

,0
TRAIL_NO,TRAIL_NO
TRAIL_NAME,TRAIL_NAME
TRAIL_TYPE,TRAIL_TYPE
TRAIL_CN,TRAIL_CN
BMP,BMP
...,...
SNOW_MOTORIZED,SNOW_MOTORIZED
WATER_MOTORIZED,WATER_MOTORIZED
OBJECTID,OBJECTID
SHAPE_Length,SHAPE_Length


In [48]:
null_df = usfs_trl_df.notna().all().to_frame()
null_df[null_df[0].notnull()]

,0
TRAIL_NO,True
TRAIL_NAME,False
TRAIL_TYPE,True
TRAIL_CN,True
BMP,True
...,...
SNOW_MOTORIZED,True
WATER_MOTORIZED,True
OBJECTID,True
SHAPE_Length,True
